# Agenda
---
* 15min: Explanation of phenotype variants, genotype variants, and their associations
* 30 min: General overview of typical GWAS pipeineline
  1. Data wrangling- QC
  2. Data Analysis/Modeling - chi squared, Odds Ratio, Linear Regression (expression data)
  3. Interpretation and visualization of p-values
     * Manhattan plots (contrasts the $−log_{10}(p-value)$ of each SNP against its genomic location, with the $\alpha=5.0×10^{−8}$ genome-wide significance line) 
     * QQplots (the quantiles of the observed p-values against those of Unif(0,1), on the −log_10 scale.)
     * $\alpha=5.0×10^{−8}$ is typical for GWAS (compared to 0.05)
* 45 min: _TAS2R38_ GWAS activity
* Basic associations (our Model & Analysis): Odds Ratio, chi-squared tests, case-control

## References
* [General GWAS pipeline](https://pmc.ncbi.nlm.nih.gov/articles/PMC10940486)
* [P-values for GWAS](https://onlinelibrary.wiley.com/doi/10.1002/gepi.20297)

# Overview of GWAS
---
## Genome Wide Association Studies illustrate all aspects of Data Science:
  
1. complex, large-scale and high-dimensionsal databases
2. practically important/relevant
3. data wrangling is crucial!
4. demonstration of a variety of statistical techniques (Odds Ratio, chi-squared test or even linear regression, for eQTL)
5. understanding the limits of your model (one example: association by linkage disequilibrium, NOT causality)
6. ethics of sensitive information 
    (tangent: [a short course on eugenics](https://qubeshub.org/community/groups/coursesource/publications?id=5101&tab_active=about&v=1))

## Genetic Variants
The genetic variants that drive "Mendelian traits" are often straight forward to categorize. 
* Think: Punnett Squares
* Examples: earwax type, cystic fibrosis, sickle cell anemia

However, **traits that result from the contribution of many variants, each of small effect, need to be resolved with statistics and big data.**
* Examples: height, Coronary Artery Disease, Type II diabetes

## What __are__ Genome-Wide Association Studies?
GWAS answer questions about genetic variants that are associated with complex traits:

> ***Is a particular genomic variant (i.e. a Single Polymorphic Trait - "SNP") associated with a particular phenotype (i.e. condition or a disease)?***

<br>

**Example Conditions:** continuous blood pressure, binary trait of high/low blood pressure

**Variant:** Usually bi-allelic SNP

> **Important note: The SNP may be responsible for the phenotypic difference, but USUALLY we assume that the SNP is simply located close to the causal variant and that it is close enough to NOT have been broken up through crossing over events yet.** It is a proxy which makes it vulnerable to violations of assumptions (Wrangling is very important). 

<br>

GWAS are also used in prioritizing variants in Pharmacogenomics. The article [Genome Wide Association Studies in Pharmacogenomics](https://ascpt.onlinelibrary.wiley.com/doi/10.1002/cpt.2349) has some illuminative figures.

### Domain Knowledge Resources:
1. "Useful Genetics" - Rosie Redfield
    * [5D- Genome-wide association studies, part 1](https://www.youtube.com/watch?v=bNpYzOr7I94)
    * [5D- Genome-wide association studies, part 2](https://www.youtube.com/watch?v=OASrRPcdM_0)
2. [(Slightly more in depth explanation)](https://www.youtube.com/watch?v=sOP8WacfBM8&t=217s)
3. [Example GWAS with Plink (software, independent of Python, that is focused on genomic analysis, such as GWAS)](https://www.youtube.com/watch?v=7QMSZx3io-Q&t=1s)
4. [Quick two page overview of GWAS](https://www.cell.com/current-biology/pdf/S0960-9822(13)00075-4.pdf)
5. [Even shorter GWAS Overview](https://www.yourgenome.org/theme/genome-wide-association-studies/)
6. [NHGRI GWAS Fact Sheet](https://www.genome.gov/about-genomics/fact-sheets/Genome-Wide-Association-Studies-Fact-Sheet)
   

## Overall Strategy of GWAS
1. Find many thousands of people who differ in the trait of interest
2. Use some kind of DNA chip (or other) to genotype their alleles at each of $10^7$ locations
3. Identify SNPs where the two phenotypic categories (very tall versus very short) have different allele frequencies

The images below help illustrate this process.
* First cartoon is a screenshot from Roside Redfield's Useful Genetics linked in the cell above
* Second is from Your Genome (the "even shorter" link in cell above)
* Third cartoon is from the NHGRI Fact Sheet

![](https://raw.githubusercontent.com/awnorowski/BDSiC_2025/refs/heads/main/images/GWASStrategyRedfield.png)

![](https://www.yourgenome.org/wp-content/uploads/2023/11/161-2015-03-12_genome-association-studies-yourgenome_CS_ES-768x461.png)

![](https://www.genome.gov/sites/default/files/inline-images/GWAS_Fact-sheet2020.jpg)

# DS Pipeline for GWAS
---
1. __Data__
   
   1. Datasets include:
       1. Genomic variation: microarrays (common variants), WGS (can include rare variants)
       2. Phenotypic variation
   2. Data sources include:
       * Biobanks, cohorts with disease-focused or population-based recruitment, direct consumer studies
        * Example: GWAS Catalogue
           * We will investigate this in 5c, and if you are interested, you can check out the summary statistics page which is useful and will give you tsv files that contain Odds Ratios and p-values) <br><br>
    
2. __Quality Control: Wrangling the Data__

    Each particular dataset and question you investigate will require a pipeline to address unique challenges that arise from unique aspects of the data/question. Genomic databases are a great example of a specialized pipeline to reduce the bias of non-independence on top of the more typical missing or inaccurate data grooming: 

    1. removing rare or monomorphic variants - minor allele frequency
    2. filtering missing SNPs 
    3. identifying and removing genotyping errors
        * heterozygosity
        * sex discrepancy
        * removing variants not in Hardy-Weinberg equilibrium (can indicate: genotyping errors, batch effects, population stratification)
        * Use PCA - a straightforward way to identify population stratification or batch effects
    4. accounting for ancestry and relatedness - another way population stratification pops up! Cases and controls should be matched by ancestry to avoid confounding and, therefore, false positives. 
    5. account for false positives (you are testing millions of hypotheses simultaneously)  <br><br>

    For your own information (we will not use this in this overview): The program Plink is standard for analysis GWAS and it requires files of specific formats: bed, fam, bim files all contain slightly different information, including genomic variants, and family relationships. **Plink makes the quality control step easier (a huge challenge) with built-in functions for the above.** There are python packages that wrap around Plink and multivariate analysis tools to create a seamless pipeline. 

3. __Analysis:__
   Common models/analysis for GWAS:
   
    1. Odds ratio
    2. Chi-squared tests
    3. Regression:
        * linear or logistic multivariate regression (eQTL)
        * generalized linear mixed-effect models
        * account for covariates: sex, age, ancestry <br><br>

5. __Visualization__ 
    * p-value Manhattan plots
    * QQplots

# _TAS2R38_ GWAS Activity
---
This activity was adapted from the The Jackson Laboratory's [Teaching the Genome Generation](https://www.jax.org/education-and-learning/high-school-students-and-undergraduates/teaching-the-genome-generation) curriculum.

## The _TAS2R38_ Gene Activity Overview
* Work through the calculations involved in the association between bitter taste perception and variants in the _TAS2R38_ gene.
* Taste 2 Receptor Member 38 produces a protein that is responsible for bitter taste (especially on the tongue) 
* Categorical phenotypes: Non-taster, Taster, Super Tasteres
* There are three SNPs that are common variants in the _TAS2R38_ gene: SNP1, SNP2, SNP3

### Group Activity

1. Split into groups of 3 (~5 groups) and you will be given the information for one of the 3 SNPs.

**SNP1** <br>
![](https://raw.githubusercontent.com/awnorowski/BDSiC_2025/refs/heads/main/images/GWAS_SNP1.png) <br><br>
**SNP2** <br>
![](https://raw.githubusercontent.com/awnorowski/BDSiC_2025/refs/heads/main/images/GWAS_SNP2.png) <br><br>
**SNP3** <br>
![](https://raw.githubusercontent.com/awnorowski/BDSiC_2025/refs/heads/main/images/GWAS_SNP3.png) <br><br>

2. (10 minutes) Count the number of allele variants associated with each of the three phenotypes for your particular SNP.
   
3. (5 minutes) We will then come back together as a class and discuss which variants - if any - of SNP1, SNP2, and SNP3 are associated with the three classes of tasting phenotype.

4. (10 minutes) Discussion of how we would *quantify* evidence for how variants might be associated with a phenotype?

5. (20 minutes) $\chi^2$ test and Odds Ratio
    1. First, focus on your SNP (1, 2, or 3) and calculate **expected count** of each of the two variants _if the variant was not associated with a particular phenotype_.
        * Note: this is the NULL hypothesis of _NO ASSOCIATION_ which is what we are testing.
        * **Equation:** Expected count = Allele freq of allele in total pop * total Allele count for specific phenotype
    2. Then, calculate **$\chi^2$**
        * **General Equation:**  $\chi^2 = \sum_{i=1}^{k}\frac{(x_i-m_i)^2}{m_i}$
        * There are two alleles per SNP so, the equation will be: $\chi^2 = \frac{(O_1-E_1)^2}{E_1} + \frac{(O_2-E_2)^2}{E_2}$
            * For this example, you are investigating one SNP so it is reasonable to do it by hand. However, GWAS looks at 10,000,000s of SNPs simultaneously, a scale that makes computation more challenging *and* introduces some novel problems (multiple testing means lots of false positives!)
            * There are built in functions to do these calculations for you!

# Data Analysis Practice

---

## We will do the following: 
1. Data -- We will look at some in module 5c when we build Manhattan plots
2. Wrangling/Grooming - Module 6B when we investigate general quality control tactics
3. Analysis - we will work through pen-and-paper example for $\chi^2$ test and for Odds ratio. In module 5B will cover regression.
4. Visualization - Module 5C Manhattan plots (and QQ plots)

## Analysis/Model - $\chi^2$ test pipeline: 
1. State your null hypothesis
2. Pick your test; make sure assumptions are valid for your data; conduct your test
3. evidence level of your test (what is $\alpha$? What is p-value, if program calculates it for you)
4. Can you Reject or Fail-to-Reject your null hypothesis?

[Here is a review of $\chi^2$](https://www.geeksforgeeks.org/python-pearsons-chi-square-test/)

__After using $\chi^2$ we are only able to reject or Fail to reject the null hypothesis, but that isn't very satisfying, is it?__

## Analysis/Model - Odds Ratio
If we have two categories that each have two categories, we can use the Odds Ratio to *quantify* the strength of an association instead of just getting a R/FTR, we can now also state magnitude of effect.  

_ODDS RATIO_: Now we want to quantify the _magnitude of the association between two categorical variables that each have two categories_

- often used in __case-control groups__
- usually testing $H_{0}$ = 1 which means that the odds of having the disease given you have the AT variant is 1 (no inflated odds)

Here is the [link to wikipedia](https://en.wikipedia.org/wiki/Odds_ratio) that explains the basics of OR. 

Here is an [example](https://rowannicholls.github.io/python/statistics/risk/odds_ratio.html) of how you would take the wikipedia example, diseases in people exposed to a radiation leak and in people who were not exposed to a radiation leak. 

>"Suppose a radiation leak in a village of 1,000 people increased the incidence of a rare disease. The total number of people exposed to the radiation was 400, out of which 20 developed the disease and 380 stayed healthy. The total number of people not exposed was 600, out of which 6 developed the disease and 594 stayed healthy."

We want to input a table that looks like this: 

|Exposed\Diseased| True | False|
|---| ---| ---|
|True|20|380|
|False|6|594|



In [1]:
import pandas as pd
import numpy as np
import scipy.stats 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
# There is another example of the same thing, here: https://rowannicholls.github.io/python/statistics/risk/odds_ratio.html
# The example from the above website shows you a more complicated way to create this table
# that gives you all 1000 data points but shows you some methods that might be useful. 
# We will use simpler code for the following example: 

# Step 1: Define the 2x2 contingency table
table_wiki = [[20, 380],
         [6, 594]]

# Create a DataFrame with headings and row names
df_wiki = pd.DataFrame(table_wiki, 
                  columns=['Diseased', 'Not Diseased'], 
                  index=['Exposed', 'Not Exposed'])

df_wiki

,Diseased,Not Diseased
Exposed,20,380
Not Exposed,6,594


In [24]:
# Risk: The risk of developing the disease given exposure, and of developing the disease given non-exposure, 
# is equal to the number of people who became diseased (20 and 6) divided by the total number of people who 
# were exposed or not exposed (400 and 600), respectively. In other words, we need to sum the values in the 
# contingency table’s rows and divide the contingency table by those values.

# Step 2: Extract the individual four cell values
a = df_wiki.loc['Exposed', 'Diseased']
b = df_wiki.loc['Exposed', 'Not Diseased']
c = df_wiki.loc['Not Exposed', 'Diseased']
d = df_wiki.loc['Not Exposed', 'Not Diseased']


# Step 3A: Calculate individual Risk 
# Get the total of each row, axis=1
totals = df_wiki.sum(axis=1)
print(totals)
print("********************")
# Divide the values in the dataframe by the total of their row
# risk is 20/(380+20)
risk = df_wiki.div(totals, axis=0)
print("risk")
print(risk)
print("********************")
#The relative risk of developing the disease given expose vs non-exposure is simply one risk value divided by another: 0.05/0.01

# Step 3b: Calculate Relative Risk
risk_exposed = a / (a + b)
risk_unexposed = c / (c + d)
relative_risk = risk_exposed / risk_unexposed
print("~~~~~~~~~~~~~~~~~~")
print(f"Risk in Exposed group: {risk_exposed:.4f}")
print(f"Risk in Unexposed group: {risk_unexposed:.4f}")
print(f"Relative Risk (RR): {relative_risk:.3f}")


Exposed        400
Not Exposed    600
dtype: int64
********************
risk
             Diseased  Not Diseased
Exposed          0.05          0.95
Not Exposed      0.01          0.99
********************
~~~~~~~~~~~~~~~~~~
Risk in Exposed group: 0.0500
Risk in Unexposed group: 0.0100
Relative Risk (RR): 5.000


In [27]:
# Step 4: Odds and Odds Ratio
# odds: The odds of getting the disease if exposed is the ratio of the number of people that became diseased to the number
# that did not - ie 20 divided by 380 - and similar for those who were not exposed:
odds_exposed = a/b
print("odds for exposed",round(odds_exposed,2))
odds_unexposed = c/d
print("odds for exposed",round(odds_unexposed,2))
# Odds Ratio
odds_ratio = odds_exposed / odds_unexposed
print("Odds Ratio: ", round(odds_ratio,1))

# Step 5: if you want to get a p-value for this, use a fisher's exact test from scipy.stats
print(scipy.stats.fisher_exact(table_wiki))
# you can see the pvalue is 0.000146 and the odds ratio is the same, which is comforting!

odds for exposed 0.05
odds for exposed 0.01
Odds Ratio:  5.2
SignificanceResult(statistic=5.2105263157894735, pvalue=0.000145624155415758)


In [29]:
# CASE-CONTROL EXAMPLE:
# The Wikipedia page goes on to give a second example wherein the data from all 26 diseased villagers
# is included but only that from 26 of the healthy villagers is available (which is a more realistic scenario):
# same data wrangling as in the above example, but now we CAN'T compute relative risk because we don't know the true
# exposed/unexposed in the population. 

# Create a data frame from a dictionary
dct = {
    'exposed': [True] * 30 + [False] * 22,
    'diseased': [True] * 20 + [False] * 10 + [True] * 6 + [False] * 16,
}
df = pd.DataFrame(dct)
contingency_table = pd.crosstab(df['exposed'], df['diseased'])
# Optional: we could have done this via this method: reverse index and columns for presentation (matches Wikipedia formatting)
# contingency = contingency.iloc[::-1, ::-1]

print("Case control contingency table")
print(contingency_table)
# The relative risk cannot be calculated because we don’t have data from the entire population,
# but we can get the odds ratio by follow the same steps as above:
odds = contingency_table[True] / contingency_table[False]
odds_ratio = odds[True] / odds[False]
print("Case-Control Odds Ratio: ")
print("Odds Ratio: ", round(odds_ratio,1))

Case control contingency table
diseased  False  True 
exposed               
False        16      6
True         10     20
Case-Control Odds Ratio: 
Odds Ratio:  5.3
